In [1]:
import sys

sys.path.append('../')

import bz2
import csv, logging, pickle5 as pickle, pandas as pd, re, random
from dewiki.parser import Parser

from wikitextprocessor import Wtp
import seaborn as sns

import requests
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

import requests, os


def pickler(path, pkl_name, obj):
    with open(os.path.join(path, pkl_name), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def unpickler(path, pkl_name):
    with open(os.path.join(path, pkl_name), 'rb') as f:
        obj = pickle.load(f)
    return obj


In [7]:
def get_labels(data):
    labels = []
    for d in data:
        labels.append(d['label']) if d['label'] not in labels else None

    return labels

def get_x_and_y(data):
    x, y = [], []
    for d in data:
        for text in d['data']:
            tmp = text['text'].replace('\n', '').replace('_', '')  # clean
            x.append(tmp) if len(tmp) > 0 else None
            y.append(d['label']) if len(tmp) > 0 else None

    return x, y

In [8]:
data = unpickler('data','topic_texts_data_markup_100_v3.pkl')

x, y = get_x_and_y(data)  # starte

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)



In [8]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(x_train, y=y_train)
result_labels = model.predict(x_test)

mat = confusion_matrix(y_test, result_labels)

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=get_labels(data), yticklabels=get_labels(data), linewidths=.5)

print('Accuracy score: %.2f' % accuracy_score(result_labels, y_test))
print(classification_report(y_test, result_labels))
print(mat)

Accuracy score: 0.52


/home/cnytync/Dev/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cnytync/Dev/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cnytync/Dev/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

                              precision    recall  f1-score   support

                Architecture       0.59      0.78      0.67        63
      Architecture-Structure       0.97      0.56      0.71        64
                  Art-Cinema       0.63      0.34      0.44        65
            Art-Cinema-Actor       0.25      0.04      0.07        68
                   Art-Dance       0.00      0.00      0.00        42
            Art-Dance-Dancer       0.40      0.57      0.47        54
                 Art-Fashion       0.72      0.60      0.66        65
        Art-Fashion-Designer       0.84      0.56      0.67        68
           Art-Fashion-Model       0.00      0.00      0.00         7
              Art-Literature       0.61      0.35      0.45        65
       Art-Literature-Writer       0.60      0.29      0.39        63
                   Art-Music       0.58      0.98      0.73        55
        Art-Music-Instrument       0.96      0.72      0.82        64
                Art

In [9]:
model = make_pipeline(TfidfVectorizer(), LinearSVC())
model.fit(x_train, y=y_train)
result_labels = model.predict(x_test)

mat = confusion_matrix(y_test, result_labels)

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=get_labels(data), yticklabels=get_labels(data), linewidths=.5)

print('Accuracy score: %.2f' % accuracy_score(result_labels, y_test))
print(classification_report(y_test, result_labels))
print(mat)

Accuracy score: 0.72


/home/cnytync/Dev/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cnytync/Dev/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cnytync/Dev/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

                              precision    recall  f1-score   support

                Architecture       0.89      0.89      0.89        38
      Architecture-Structure       0.68      0.89      0.77        28
                  Art-Cinema       0.73      0.76      0.75        29
            Art-Cinema-Actor       0.50      0.44      0.47        32
                   Art-Dance       0.86      0.86      0.86        29
            Art-Dance-Dancer       0.81      0.69      0.75        32
                 Art-Fashion       0.68      0.72      0.70        32
        Art-Fashion-Designer       0.62      0.53      0.57        34
           Art-Fashion-Model       1.00      0.20      0.33        15
              Art-Literature       0.76      0.79      0.78        33
       Art-Literature-Writer       0.59      0.54      0.57        35
                   Art-Music       0.78      0.92      0.85        39
        Art-Music-Instrument       0.90      0.84      0.87        32
                Art

In [37]:
model = make_pipeline(TfidfVectorizer(), DecisionTreeClassifier())
model.fit(x_train, y=y_train)
result_labels = model.predict(x_test)

mat = confusion_matrix(y_test, result_labels)

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=get_labels(data), yticklabels=get_labels(data), linewidths=.5)


print('Accuracy score: %.2f' % accuracy_score(result_labels, y_test))
print(classification_report(y_test, result_labels))
print(mat)

Accuracy score: 0.48
                              precision    recall  f1-score   support

                Architecture       0.65      0.76      0.70        37
      Architecture-Structure       0.52      0.45      0.48        29
                  Art-Cinema       0.62      0.69      0.66        29
            Art-Cinema-Actor       0.16      0.12      0.14        32
                   Art-Dance       0.48      0.38      0.42        29
            Art-Dance-Dancer       0.39      0.39      0.39        33
                 Art-Fashion       0.55      0.56      0.55        32
        Art-Fashion-Designer       0.56      0.58      0.57        33
           Art-Fashion-Model       0.78      0.47      0.58        15
              Art-Literature       0.44      0.36      0.40        33
       Art-Literature-Writer       0.19      0.17      0.18        36
                   Art-Music       0.77      0.69      0.73        39
        Art-Music-Instrument       0.44      0.56      0.49        3

In [42]:
model = make_pipeline(TfidfVectorizer(), MLPClassifier())
model.fit(x_train, y=y_train)
result_labels = model.predict(x_test)

mat = confusion_matrix(y_test, result_labels)

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=get_labels(data), yticklabels=get_labels(data), linewidths=.5)


print('Accuracy score: %.2f' % accuracy_score(result_labels, y_test))
print(classification_report(y_test, result_labels))
print(mat)

/home/cnytync/Dev/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy score: 0.01
                              precision    recall  f1-score   support

                Architecture       0.00      0.00      0.00        63
      Architecture-Structure       0.00      0.00      0.00        64
                  Art-Cinema       0.00      0.00      0.00        65
            Art-Cinema-Actor       0.00      0.00      0.00        68
                   Art-Dance       0.00      0.00      0.00        42
            Art-Dance-Dancer       0.00      0.00      0.00        54
                 Art-Fashion       1.00      0.02      0.03        65
        Art-Fashion-Designer       0.00      0.00      0.00        68
           Art-Fashion-Model       0.00      0.00      0.00         7
              Art-Literature       0.00      0.00      0.00        65
       Art-Literature-Writer       0.00      0.00      0.00        63
                   Art-Music       0.00      0.00      0.00        55
        Art-Music-Instrument       0.00      0.00      0.00        6

/home/cnytync/Dev/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cnytync/Dev/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cnytync/Dev/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TENSORFLOR DNN

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import losses

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import pickle5 as pickle

def unpickler(path, pkl_name):
    with open(os.path.join(path, pkl_name), 'rb') as f:
        obj = pickle.load(f)
    return obj

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cnytync/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def get_label_index(label):
    return [index for index, _label in enumerate(labels) if label == _label][0]

def get_labels(data):
    labels = []
    for d in data:
        labels.append(d['label']) if d['label'] not in labels else None

    return labels

def get_subject_index(subject):
    if not subject: return 0
    return [index for index, _subject in enumerate(subjects) if subject == _subject][0]

def get_subjects(data):
    subjects = []
    subjects.append('None')
    
    for d in data:
        for dd in d['data']:
            subjects.append(dd['subject']) if dd['subject'] and dd['subject'] not in subjects else None
    
    return subjects

def get_x_and_y(data):
    x, y = [], []
    for d in data:
        for dd in d['data']:
            tmp = dd['text'].replace('\n', '').replace('_', '')  # clean
            x.append({'label': d['label'], 'subject': dd['subject'], 'dbpedia_uri': dd['dbpedia_uri'], 'text': tmp, 'graph': dd['graph']}) if len(tmp) > 0 else None
            y.append(d['label']) if len(tmp) > 0 else None

    return x, y

def join_x_and_y(x,y):
    return [(_x,_y) for _x, _y in zip(x,y)]

In [3]:
data = unpickler('data','classification_data_with_graphs_v4.pkl')

labels = get_labels(data)
subjects = get_subjects(data)

x,y = get_x_and_y(data)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# x_train_subject, x_test_subject = [get_subject_index(xx['subject']) for xx in x_train], [get_subject_index(xx['subject']) for xx in x_test]
x_train_subject, x_test_subject = ['None' if not xx['subject'] else xx['subject'] for xx in x_train], ['None' if not xx['subject'] else xx['subject']  for xx in x_test]
x_train_text,x_test_text = [xx['text'] for xx in x_train], [xx['text'] for xx in x_test]
y_train_int = [get_label_index(label) for label in y_train]


data_train = join_x_and_y(x_train, y_train)
data_test = join_x_and_y(x_test, y_test)

In [4]:
max_features = 10000
sequence_length = 200

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    stripped_html = tf.strings.regex_replace(stripped_html,r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', ' ')
    stripped_html = tf.strings.regex_replace(stripped_html, r'@([A-Za-z0-9_]+)', ' ' )
    for i in stopwords.words('english'):
        stripped_html = tf.strings.regex_replace(stripped_html, f' {i} ', " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    # standardize=custom_standardization,
    output_mode='int',
    output_sequence_length=sequence_length
)

with tf.device("/CPU:0"):
    vectorize_layer.adapt(x_train_text)

#### SIMPLE DNN

In [5]:
MAX_TOKENS_NUM = 5000  # Maximum vocab size.
MAX_SEQUENCE_LEN = 40  # Sequence length to pad the outputs to.
EMBEDDING_DIMS = 100

model_simple = tf.keras.models.Sequential()
model_simple.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model_simple.add(vectorize_layer)

model_simple.add(tf.keras.layers.Embedding(MAX_TOKENS_NUM + 1, EMBEDDING_DIMS))
model_simple.add(layers.Dropout(0.1))
model_simple.add(tf.keras.layers.GlobalAveragePooling1D())
model_simple.add(layers.Dropout(0.1))


model_simple.add(layers.Dense(len(labels)))
model_simple.summary()
model_simple.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.SparseCategoricalAccuracy())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 200)               0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 100)          500100    
_________________________________________________________________
dropout (Dropout)            (None, 200, 100)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 78)                7878      
Total params: 507,978
Trainable params: 507,978
Non-trainable params: 0
__________________________________________________

In [5]:
MAX_TOKENS_NUM = 5000  # Maximum vocab size.
MAX_SEQUENCE_LEN = 40  # Sequence length to pad the outputs to.
EMBEDDING_DIMS = 100

text_input = tf.keras.Input(shape=(1,), dtype=tf.string)

text_layer = vectorize_layer(text_input)
text_layer = tf.keras.layers.Embedding(MAX_TOKENS_NUM + 1, EMBEDDING_DIMS)(text_layer)
text_layer = tf.keras.layers.GlobalAveragePooling1D()(text_layer)

final_output = tf.keras.layers.Dense(units=len(labels))(text_layer)

model_simple = tf.keras.models.Model(inputs=[text_input], outputs=final_output)
model_simple.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.SparseCategoricalAccuracy())

In [6]:
i = 12
x_train_text[i], labels[y_train_int[i]]

('Anthropology of religion is the study of religion in relation to other social institutions, and the comparison of religious beliefs and practices across cultures.HistoryAl-Biruni (973–1048), wrote detailed comparative studies on the anthropology of religions and cultures across the Mediterranean Basin (including the so-called  "Middle East") and the Indian subcontinent. He discussed the peoples, customs, and religions of the Indian subcontinent.In the 19th century cultural anthropology was dominated by an interest in cultural evolution; most anthropologists assumed a simple distinction between "primitive" and "modern" religion and tried to provide accounts of how the former evolved into the latter.In 1912 Émile Durkheim, building on the work of Feuerbach, considered religion "a projection of the social values of society", "a means of making symbolic statements about society", "a symbolic language that makes statements about the social order";Anthropologists circa 1940 assumed that re

In [7]:
epochs =  10
history = model_simple.fit(
    x_train_text,
    y=y_train_int,
    epochs=epochs)

Epoch 1/10
141/141 [==============================] - ETA: 1:35 - loss: 4.3606 - sparse_categorical_accuracy: 0.0000e+0 - ETA: 1s - loss: 4.3560 - sparse_categorical_accuracy: 0.0208      - ETA: 1s - loss: 4.3548 - sparse_categorical_accuracy: 0.019 - ETA: 1s - loss: 4.3533 - sparse_categorical_accuracy: 0.027 - ETA: 1s - loss: 4.3525 - sparse_categorical_accuracy: 0.034 - ETA: 1s - loss: 4.3513 - sparse_categorical_accuracy: 0.033 - ETA: 1s - loss: 4.3508 - sparse_categorical_accuracy: 0.031 - ETA: 1s - loss: 4.3492 - sparse_categorical_accuracy: 0.033 - ETA: 1s - loss: 4.3479 - sparse_categorical_accuracy: 0.033 - ETA: 1s - loss: 4.3470 - sparse_categorical_accuracy: 0.034 - ETA: 1s - loss: 4.3459 - sparse_categorical_accuracy: 0.038 - ETA: 0s - loss: 4.3451 - sparse_categorical_accuracy: 0.038 - ETA: 0s - loss: 4.3435 - sparse_categorical_accuracy: 0.041 - ETA: 0s - loss: 4.3415 - sparse_categorical_accuracy: 0.048 - ETA: 0s - loss: 4.3408 - sparse_categorical_accuracy: 0.049 - ETA:

In [13]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.show()
plt.plot(history.history['loss'])
plt.show()

In [14]:
results = model_simple.predict(x_test_text)
def get_result_labels(results):
    return [labels[np.where(row==max(row))[0][0]] for row in results]

result_labels = get_result_labels(results)
print('Accuracy score: %.2f' % accuracy_score(result_labels, y_test))
print(classification_report(y_test, result_labels))


Accuracy score: 0.61
                              precision    recall  f1-score   support

                Architecture       0.88      0.81      0.84        36
      Architecture-Structure       0.85      0.63      0.72        27
                  Art-Cinema       0.45      0.76      0.56        29
            Art-Cinema-Actor       0.32      0.22      0.26        32
                   Art-Dance       0.77      0.68      0.72        25
            Art-Dance-Dancer       0.62      0.56      0.59        32
                 Art-Fashion       0.57      0.61      0.59        33
        Art-Fashion-Designer       0.68      0.38      0.49        34
           Art-Fashion-Model       1.00      0.29      0.44        14
              Art-Literature       0.67      0.50      0.57        32
       Art-Literature-Writer       0.54      0.43      0.48        35
                   Art-Music       0.76      0.87      0.81        39
        Art-Music-Instrument       0.92      0.73      0.81        3

#### DNN WITH SUBJECTS

In [12]:
MAX_TOKENS_NUM = 5000  # Maximum vocab size.
MAX_SEQUENCE_LEN = 40  # Sequence length to pad the outputs to.
EMBEDDING_DIMS = 100


text_input = tf.keras.Input(shape=(1,), dtype=tf.string)
subject_input = tf.keras.Input(shape=(1,), dtype=tf.string)

text_layer = vectorize_layer(text_input)
text_layer = tf.keras.layers.Embedding(MAX_TOKENS_NUM + 1, EMBEDDING_DIMS)(text_layer)
text_layer = tf.keras.layers.GlobalAveragePooling1D()(text_layer)

subject_layer = tf.keras.layers.StringLookup(vocabulary=subjects, output_mode='one_hot')(subject_input)
subject_layer = tf.keras.layers.Lambda(lambda x: x * 100)(subject_layer)

output = text_layer

model = tf.keras.models.Model(inputs=text_input, outputs=output)

res = model.predict(x_train_text[:2])
sum(res[0]),sum(res[1])

model2 = tf.keras.models.Model(inputs=subject_input, outputs=subject_layer)
model2.predict(x_train_subject[:2])

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0., 100.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 

In [6]:
MAX_TOKENS_NUM = 5000  # Maximum vocab size.
MAX_SEQUENCE_LEN = 40  # Sequence length to pad the outputs to.
EMBEDDING_DIMS = 100

text_input = tf.keras.Input(shape=(1,), dtype=tf.string)
subject_input = tf.keras.Input(shape=(1,), dtype=tf.string)

text_layer = vectorize_layer(text_input)
text_layer = tf.keras.layers.Embedding(MAX_TOKENS_NUM + 1, EMBEDDING_DIMS)(text_layer)
text_layer = tf.keras.layers.GlobalAveragePooling1D()(text_layer)

subject_layer = tf.keras.layers.StringLookup(vocabulary=subjects, output_mode='one_hot')(subject_input)
# subject_layer = tf.keras.layers.Lambda(lambda x: x * 100)(subject_layer)

concatenated = tf.keras.layers.Concatenate(axis=1)([text_layer, subject_layer])

output = tf.keras.layers.Dense(len(labels))(concatenated)

model = tf.keras.models.Model(inputs=[text_input, subject_input], outputs=output)

model.summary()
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.SparseCategoricalAccuracy())

# tf.keras.utils.plot_model(model, to_file='model.png')


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
text_vectorization_1 (TextVecto (None, 200)          0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 100)     500100      text_vectorization_1[0][0]       
______________________________________________________________________________________________

In [7]:
epochs = 60
history = model.fit(
    [pd.DataFrame(x_train_text).values,pd.DataFrame(x_train_subject).values],
    y=pd.DataFrame(y_train_int).values,
    epochs=epochs)

Epoch 1/100
141/141 [==============================] - ETA: 1:37 - loss: 13.2699 - sparse_categorical_accuracy: 0.0000e+ - ETA: 1s - loss: 12.8822 - sparse_categorical_accuracy: 0.0063     - ETA: 1s - loss: 12.1405 - sparse_categorical_accuracy: 0.00 - ETA: 1s - loss: 11.7494 - sparse_categorical_accuracy: 0.01 - ETA: 1s - loss: 11.4032 - sparse_categorical_accuracy: 0.01 - ETA: 1s - loss: 11.1805 - sparse_categorical_accuracy: 0.02 - ETA: 1s - loss: 11.1408 - sparse_categorical_accuracy: 0.02 - ETA: 1s - loss: 10.8988 - sparse_categorical_accuracy: 0.02 - ETA: 1s - loss: 10.8608 - sparse_categorical_accuracy: 0.02 - ETA: 1s - loss: 10.6638 - sparse_categorical_accuracy: 0.03 - ETA: 1s - loss: 10.5160 - sparse_categorical_accuracy: 0.03 - ETA: 1s - loss: 10.2924 - sparse_categorical_accuracy: 0.03 - ETA: 1s - loss: 10.1058 - sparse_categorical_accuracy: 0.03 - ETA: 0s - loss: 9.9808 - sparse_categorical_accuracy: 0.0349 - ETA: 0s - loss: 9.9025 - sparse_categorical_accuracy: 0.037 - ET

In [8]:
plt.plot(history.history['loss'])
plt.show()
plt.plot(history.history['sparse_categorical_accuracy'])
plt.show()

In [9]:
results = model.predict([pd.DataFrame(x_test_text).values, pd.DataFrame(x_test_subject).values])
def get_result_labels(results):
    return [labels[np.where(row==max(row))[0][0]] for row in results]

result_labels = get_result_labels(results)
print('Accuracy score: %.2f' % accuracy_score(result_labels, y_test))
print(classification_report(y_test, result_labels))

Accuracy score: 0.66


/home/cnytync/Dev/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cnytync/Dev/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cnytync/Dev/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

                              precision    recall  f1-score   support

                Architecture       0.91      0.89      0.90        36
      Architecture-Structure       1.00      0.96      0.98        27
                  Art-Cinema       0.76      0.86      0.81        29
            Art-Cinema-Actor       0.43      0.19      0.26        32
                   Art-Dance       0.96      0.92      0.94        25
            Art-Dance-Dancer       0.76      0.59      0.67        32
                 Art-Fashion       0.66      0.58      0.61        33
        Art-Fashion-Designer       0.56      0.59      0.57        34
           Art-Fashion-Model       0.67      0.29      0.40        14
              Art-Literature       0.82      0.88      0.85        32
       Art-Literature-Writer       0.53      0.69      0.60        35
                   Art-Music       0.97      0.97      0.97        39
        Art-Music-Instrument       0.94      0.88      0.91        33
                Art